<a href="https://colab.research.google.com/github/AmiraOsamaMohamed/GraduationProject/blob/main/GProject_ipyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [50]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import nltk
import re
import spacy
from nltk import word_tokenize , sent_tokenize
from nltk.corpus import stopwords

In [51]:
#read json file and convert it to DataFrame
import json
def read_data(path):
 data = []

 with open(path) as trainFile:
    for line in trainFile:
        data.append(json.loads(line))
 all_data=pd.DataFrame(data)
 all_data.head()
 #split col(answers) to text and start_char and combine them as base columns in big DataFrame
 line=[]
 list_text=[]
 list_start_char=[]
 list_pq_id=[]

 for i in range(0,len(data)):
   line.extend(data[i]['answers'])


 for i in range(0,len(data)):
   list_text.append(line[i]['text'])



 for i in range(0,len(data)):
   list_start_char.append(line[i]['start_char'])



 for i in range(0,len(data)):
   list_pq_id.append(data[i]['pq_id'])



 answer_df=pd.DataFrame(list_pq_id,columns=['pq_id'])
 answer_df['text']=list_text
 answer_df['start_char']=list_start_char
 answer_df.head()

#merge between 2 DataFrame
 merged_data = pd.merge(all_data, answer_df, on="pq_id")
 merged_data = merged_data.drop(columns=["answers"])
 merged_data = merged_data.rename(columns={"text":"answer"})
 merged_data.head()
 return merged_data

In [52]:
#convert merged data to csv file
train=read_data('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_train.jsonl')
validation=read_data('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_dev.jsonl')
test=read_data('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_test_gold.jsonl')
train.to_csv('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_train.csv', index=False)
validation.to_csv('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_dev.csv', index=False)
test.to_csv('/content/drive/MyDrive/data/QQA23_TaskB_qrcd_v1.2_test_gold.csv', index=False)

In [53]:
#clean text
def clean(text):
    text = re.sub(r"http\S+", " ", text)  # remove urls
    text = re.sub(r"@[\w]*", " ", text)  # remove handles
    text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
    text = re.sub(r"\t", " ", text)  # remove tabs
    text = re.sub(r"\n", " ", text)  # remove line jump
    text = re.sub(r"\s+", " ", text)  # remove extra white space
    text = re.sub(r'[^\w\s]', '', text) # Removing punctuations in string using regex
    text = text.strip()
    return text

In [54]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
# Function to tokenize sentences
def tokenize_words(text):
    return word_tokenize(text)

# Function to remove stopwords from tokenized text
stop_words = set(stopwords.words('arabic'))
def remove_stopwords(tokenized_text):
    return [word for word in tokenized_text if word not in stop_words]


# Function to return to the base of word(stemmer) from tokenized text
stemmer = SnowballStemmer('arabic')
def stem_words(tokenized_text):
    return [stemmer.stem(word) for word in tokenized_text]


def processed_text(text):
  cleaned_text=clean(text)
  tokenized_text=tokenize_words(cleaned_text)
  underProcessing_text=remove_stopwords(tokenized_text)
  processed_text=stem_words(underProcessing_text)
  return processed_text


# Apply the functions(processing) to the DataFrame
train['processed passage'] = train['passage'].apply(processed_text)


train['processed question'] = train['question'].apply(processed_text)


train['processed answer'] = train['answer'].apply(processed_text)



# Print the DataFrame
train.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,pq_id,passage,surah,verses,question,answer,start_char,processed passage,processed question,processed answer
0,2:1-5_372,الم. ذلك الكتاب لا ريب فيه هدى للمتقين. الذين ...,2,1-5,ما الدلائل على أن القرآن ليس من تأليف سيدنا مح...,الذين يؤمنون بما أنزل إليك وما أنزل من قبلك,97,"[الم, كتاب, ريب, هدى, متق, يومن, غيب, يقيم, صل...","[دلايل, قرء, تاليف, سيد, محمد]","[يومن, انزل, انزل, قبل]"
1,2:23-24_372,وإن كنتم في ريب مما نزلنا على عبدنا فأتوا بسور...,2,23-24,ما الدلائل على أن القرآن ليس من تأليف سيدنا مح...,إن كنتم في ريب مما نزلنا على عبدنا فأتوا بسورة...,1,"[كنتم, ريب, نزل, عبد, اتو, سور, مثل, وادع, شهد...","[دلايل, قرء, تاليف, سيد, محمد]","[كنتم, ريب, نزل, عبد, اتو, سور, مثل]"
2,2:40-48_372,يا بني إسرائيل اذكروا نعمتي التي أنعمت عليكم و...,2,40-48,ما الدلائل على أن القرآن ليس من تأليف سيدنا مح...,آمنوا بما أنزلت مصدقا لما معكم,85,"[بن, اسراييل, اذكر, نعم, انعم, عل, اوف, عهد, ا...","[دلايل, قرء, تاليف, سيد, محمد]","[امنو, انزل, مصدق, معكم]"
3,2:91-93_372,وإذا قيل لهم آمنوا بما أنزل الله قالوا نؤمن بم...,2,91-93,ما الدلائل على أن القرآن ليس من تأليف سيدنا مح...,إذا قيل لهم آمنوا بما أنزل الله,1,"[قيل, امنو, انزل, الله, قالو, نوم, انزل, عل, ي...","[دلايل, قرء, تاليف, سيد, محمد]","[قيل, امنو, انزل, الله]"
4,2:97-101_372,قل من كان عدوا لجبريل فإنه نزله على قلبك بإذن ...,2,97-101,ما الدلائل على أن القرآن ليس من تأليف سيدنا مح...,هو الحق مصدقا لما معهم,78,"[قل, عدو, جبريل, فان, نزل, قلب, باذ, الله, مصد...","[دلايل, قرء, تاليف, سيد, محمد]","[الحق, مصدق, معهم]"


In [55]:
validation['processed passage'] = validation['passage'].apply(processed_text)


validation['processed question'] = validation['question'].apply(processed_text)


validation['processed answer'] = validation['answer'].apply(processed_text)


In [56]:
test['processed passage'] = test['passage'].apply(processed_text)


test['processed question'] = test['question'].apply(processed_text)


test['processed answer'] = test['answer'].apply(processed_text)
